## 1. Libraries

In [1]:
import pandas as pd
import numpy
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation

## 2. Functions

In [2]:
# Remove punctuation
def remove_punctuation(text):
    text = text.lower()  # lowercase
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) 
    text = text.translate(translator)
    text = re.sub(r'\s+'," ", text)
    return text

In [3]:
# Define stopwords
nltk.download('stopwords')
stopwords = stopwords.words("english")

wo_stop_words = ['na','n a', 'nil', 'no', 'no comments', 'none', 'many', 'would', 'could', 'however']

for i in wo_stop_words:
    if i not in stopwords:
        stopwords.append(i)

[nltk_data] Downloading package stopwords to C:\Users\Chun
[nltk_data]     Quan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Remove stopwords, lemmatize, returns only alphabet characters

nltk.download('punkt') # Tokenizer model
nltk.download('wordnet') # Lexical database

def remove_stopwords_lemmatize(sentence):
    
    # Tokenize
    tokens = word_tokenize(sentence)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Remove very short tokens
    tokens = [word for word in tokens if len(word) > 2]

    # Only contains alphabet characters
    tokens = " ".join([word for word in tokens if word.isalpha()])
    return tokens

[nltk_data] Downloading package punkt to C:\Users\Chun
[nltk_data]     Quan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Chun
[nltk_data]     Quan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 3. Dataset

In [5]:
feedbadk_data = pd.read_excel("Simulated-Employee-Feedback.xlsx")

In [6]:
feedbadk_data.head()

,unique_identifier,employee_feedback,department
0,3565,There's a culture of blame within the company ...,Dept A
1,7323,The company's approach to feedback and perform...,Dept A
2,5008,"While page limits have been set, some departme...",Dept A
3,3460,na,Dept A
4,2179,The culture of collaboration within our team i...,Dept A


## 4. Data Preproecessing

In [7]:
# Apply fucntion, drop rows where the employee_feedback_cleaned field is empty
feedbadk_data['employee_feedback_cleaned'] = feedbadk_data['employee_feedback'].apply(remove_punctuation)
feedbadk_data['employee_feedback_cleaned'] = feedbadk_data['employee_feedback_cleaned'].apply(remove_stopwords_lemmatize)
feedbadk_data = feedbadk_data[feedbadk_data['employee_feedback_cleaned'].str.strip() != '']

## 5. Count Vectoriser

In [8]:
cv = CountVectorizer()
dtm = cv.fit_transform(feedbadk_data['employee_feedback_cleaned'])

## 6. What are the set of topics from the response text data?

In [9]:
# Latent Dirichlet Allocation
param_grid = {'n_components': [5, 10, 15, 20]}
lda = LatentDirichletAllocation(random_state=42)
grid_search = GridSearchCV(lda, param_grid=param_grid, cv=5)
grid_search.fit(dtm)

GridSearchCV(cv=5, estimator=LatentDirichletAllocation(random_state=42),
             param_grid={'n_components': [5, 10, 15, 20]})

In [10]:
# Get best model using best parameters
best_lda_model = grid_search.best_estimator_

# Extract top words for each topic from the LDA model
def get_top_words(model, feature_names, n_top_words=10):
    topic_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Topic #{topic_idx}:", top_words)
        topic_words[topic_idx] = top_words
    return topic_words

# Extract features names from the CountVectorizer
feature_names = cv.get_feature_names_out()

# print top words per topic
topic_names = get_top_words(best_lda_model, feature_names, n_top_words=10)

# Answer:
# Topic #0: ['work', 'workload', 'company', 'job', 'health', 'mental', 'need', 'staff', 'time', 'initiative'],
# intepreted as Workload, Health and Wellbeing (workload, health, mental health)
# Topic #1: ['work', 'time', 'team', 'feel', 'area', 'life', 'recognition', 'effort', 'like', 'pay'];
# intepreted as Recognition and Compensation (recognition, effort, pay)
# Topic #2: ['feedback', 'company', 'make', 'performance', 'manager', 'opportunity', 'review', 'issue', 'employee', 'team'],
# intepreted as Performance Management and Career Growth (feedback, performance, opportunity, review )
# Topic #3: ['company', 'work', 'feel', 'like', 'employee', 'appreciate', 'make', 'goal', 'support', 'team'],
# intepreted as Culture and Support (appreciate, goal, support)
# Topic #4: ['team', 'new', 'management', 'day', 'process', 'plus', 'benefit', 'etc', 'leader', 'package'],
# intepreted as Leadership and Benefits (management, process, benefit, leader, package)

Topic #0: ['work', 'workload', 'company', 'job', 'health', 'mental', 'need', 'staff', 'time', 'initiative']
Topic #1: ['work', 'time', 'team', 'feel', 'area', 'life', 'recognition', 'effort', 'like', 'pay']
Topic #2: ['feedback', 'company', 'make', 'performance', 'manager', 'opportunity', 'review', 'issue', 'employee', 'team']
Topic #3: ['company', 'work', 'feel', 'like', 'employee', 'appreciate', 'make', 'goal', 'support', 'team']
Topic #4: ['team', 'new', 'management', 'day', 'process', 'plus', 'benefit', 'etc', 'leader', 'package']


## 7. What are the set of topics concerning different departments

In [11]:
# Get topic distribution (probabilities) for each document
topic_distributions = best_lda_model.transform(dtm)
#  Assign the dominant topic (the one with highest probability) to each document
feedbadk_data['dominant_topic'] = topic_distributions.argmax(axis=1)

In [12]:
# Group data by department and dominant topic, count number of documents per group
topic_counts = feedbadk_data.groupby(['department', 'dominant_topic']).size().unstack()
print(topic_counts)

#Answer:
# Top 2 topics by Dept:
# Dept A is more concenred on topics 3 (Culture and Support) and 4 (Leadership and Benefits)
# Dept B is more concenred on topics 3 (Culture and Support) and 0 (Workload, Health and Wellbeing)
# Dept C is more concenred on topics 3 (Culture and Support) and 0 (Workload, Health and Wellbeing)
# Dept D is more concenred on topics 4 (Leadership and Benefits) and 2 (Performance Management and Career Growth)
# given:
# Topic #0: ['work', 'workload', 'company', 'job', 'health', 'mental', 'need', 'staff', 'time', 'initiative'],
# intepreted as Workload, Health and Wellbeing (workload, health, mental health)
# Topic #1: ['work', 'time', 'team', 'feel', 'area', 'life', 'recognition', 'effort', 'like', 'pay'];
# intepreted as Recognition and Compensation (recognition, effort, pay)
# Topic #2: ['feedback', 'company', 'make', 'performance', 'manager', 'opportunity', 'review', 'issue', 'employee', 'team'],
# intepreted as Performance Management and Career Growth (feedback, performance, opportunity, review )
# Topic #3: ['company', 'work', 'feel', 'like', 'employee', 'appreciate', 'make', 'goal', 'support', 'team'],
# intepreted as Culture and Support (appreciate, goal, support)
# Topic #4: ['team', 'new', 'management', 'day', 'process', 'plus', 'benefit', 'etc', 'leader', 'package'],
# intepreted as Leadership and Benefits (management, process, benefit, leader, package)

dominant_topic   0   1   2   3  4
department                       
Dept A           5   3   4  12  6
Dept B           2   1   1   6  1
Dept C          11  10  10  29  7
Dept D           5   4   7   5  7


## 8. What can we infer about the profile of individuals?

In [13]:
# Create dictionaries
topic_labels = {idx: " ".join(words) for idx, words in topic_names.items()}
topic_labels_interpretation = {
    0: 'Workload, Health and Wellbeing',
    1: 'Recognition and Compensation',
    2: 'Performance Management & Career Growth',
    3: 'Culture and Support',
    4: 'Leadership and Benefits'
}
# Mapping
feedbadk_data['dominant_topic_label_interpretation'] = feedbadk_data['dominant_topic'].map(topic_labels_interpretation)
feedbadk_data['dominant_topic_label'] = feedbadk_data['dominant_topic'].map(topic_labels)

In [14]:
feedbadk_data.sample(5)

,unique_identifier,employee_feedback,department,employee_feedback_cleaned,dominant_topic,dominant_topic_label_interpretation,dominant_topic_label
44,9411,The company's leadership team is generally eff...,Dept B,company leadership team generally effective tr...,3,Culture and Support,company work feel like employee appreciate mak...
106,5204,Job descriptions unclear or outdated,Dept C,job description unclear outdated,0,"Workload, Health and Wellbeing",work workload company job health mental need s...
81,7042,I am extremely satisfied with the level of sup...,Dept C,extremely satisfied level support receive mana...,3,Culture and Support,company work feel like employee appreciate mak...
145,7007,Our team has a shared sense of purpose. Workin...,Dept D,team shared sense purpose working towards comm...,1,Recognition and Compensation,work time team feel area life recognition effo...
4,2179,The culture of collaboration within our team i...,Dept A,culture collaboration within team truly someth...,3,Culture and Support,company work feel like employee appreciate mak...


In [15]:
# Save as excel
feedbadk_data.to_excel('Output.xlsx', index=False) # Answer